ANALIZA I PRIMENA ALATA WEIGHTS & BIASES (W&B) U PRAĆENJU ML MODELA

### Uvod

U razvoju modela mašinskog učenja, posebno u eksperimentalnim fazama, neophodno je sistematsko praćenje parametara, metrika i verzija modela. 
Ručno praćenje eksperimenata brzo postaje neefikasno i nepouzdano.

Alat Weights & Biases (W&B) predstavlja platformu za:
-praćenje eksperimenata (experiment tracking) <br>
-logovanje metrika u realnom vremenu <br>
-verzionisanje modela i podataka <br>
-vizuelizaciju trening procesa <br>
-poređenje više modela <br>

U ovom radu prikazujemo:
  
1.Proceduru instalacije i podešavanja W&B <br>
2.Povezivanje sa modelom za predikciju hepatitisa C <br>
3.Analizu funkcionalnosti <br> 
4.Poređenje sa alatom MLflow <br>

### Setup i instalacija Weights & Biases

#### Kreiranje naloga

1.Otvara se nalog na platformi https://wandb.ai<br>
2.Generiše se API ključ<br>
3.API ključ se koristi za autentifikaciju projekta

#### Instalacija
--pip install wandb nbformat scikit-learn pandas


#### Autentifikacija

Nakon instalacije:

In [18]:
import wandb
wandb.login()


True

In [ ]:
import wandb
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)

# ========== INIT RUN ==========
run = wandb.init(
    project="hepatitis_c_prediction",
    entity="milosandjelkovic20-faculty-of-natural-sciences-kragujevac",
    name="RandomForest-baseline-binary",
    config={
        "model": "RandomForest",
        "n_estimators": 200,
        "test_size": 0.20,
        "random_state": 42
    }
)

# ========== LOAD DATA ==========
df = pd.read_csv("HepatitisCdata.csv")

# drop indeks kolona
if "Unnamed: 0" in df.columns:
    df = df.drop("Unnamed: 0", axis=1)

df["Sex"] = df["Sex"].astype("category").cat.codes

# BINARIZACIJA: 0 = Blood donor, 1 = Hepatitis C
df["Category"] = df["Category"].apply(
    lambda x: 0 if "Blood Donor" in str(x) else 1
)

# ========== FEATURES / TARGET ==========
X = df.drop("Category", axis=1)
y = df["Category"]

wandb.config["features"] = list(X.columns)

# ========== TRAIN/TEST SPLIT ==========
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=run.config["test_size"],
    random_state=run.config["random_state"],
    stratify=y
)

print("Train distribution:\n", y_train.value_counts())
print("Test distribution:\n", y_test.value_counts())

# ========== TRAIN MODEL ==========
model = RandomForestClassifier(
    n_estimators=run.config["n_estimators"],
    random_state=run.config["random_state"]
)
model.fit(X_train, y_train)

# ========== PREDICT ==========
pred = model.predict(X_test)

# Pretvori u numpy da nema pandas index problema
y_true = y_test.to_numpy()
y_pred = np.asarray(pred)

# ========== METRICS ==========
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec  = recall_score(y_true, y_pred, zero_division=0)
f1   = f1_score(y_true, y_pred, zero_division=0)

cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

print("\nMetrics:")
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1:", f1)
print("\nConfusion matrix (rows=true, cols=pred):\n", cm)

wandb.log({
    "accuracy": acc,
    "precision": prec,
    "recall": rec,
    "f1_score": f1,
    "confusion_matrix": wandb.Table(
        data=cm.tolist(),
        columns=["Pred_BloodDonor", "Pred_HepatitisC"]
    )
})

wandb.finish()


Train distribution:
 Category
0    432
1     60
Name: count, dtype: int64
Test distribution:
 Category
0    108
1     15
Name: count, dtype: int64

Metrics:
Accuracy: 0.983739837398374
Precision: 1.0
Recall: 0.8666666666666667
F1: 0.9285714285714286

Confusion matrix (rows=true, cols=pred):
 [[108   0]
 [  2  13]]


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.98374
f1_score,0.92857
precision,1
recall,0.86667


<p align="center">
  <img src="runs-tab.png" width="600"/>
</p>

<p align="center">
Slika 1: Lista eksperimenata u okviru W&B projekta hepatitis_c_prediction
</p>

---

<p align="center">
  <img src="config_matrike.png" width="600"/>
</p>

<p align="center">
Slika 2: Detaljan prikaz RandomForest-baseline run-a sa logovanim hiperparametrima i metrikama
</p>

---

<p align="center">
  <img src="grafik_test_accuracy.png" width="600"/>
</p>

<p align="center">
Slika 3: Grafički prikaz vrednosti metrika
</p>


### Diskusija

U okviru projekta uspešno je demonstrirana integracija modela za klasifikaciju hepatitisa C sa platformom Weights & Biases. 
Alat omogućava automatsko logovanje hiperparametara i metrika, kao i grafički prikaz performansi modela. 
Na osnovu dobijenih rezultata može se uočiti da RandomForest model postiže visoku tačnost na test skupu, 
što potvrđuje njegovu pogodnost za ovaj tip problema.

Weights & Biases značajno olakšava eksperimentalni rad jer omogućava brzo poređenje različitih modela i konfiguracija.
